# American Option Pricing with QuantLib

In this notebok we illustrate American option pricing with QuantLib.

The notebook is structured as follows.

  1. Setup of market data and model.
  2. Pricing American call and put options.
  3. Calculation of continuation and stopping region. 

In [ ]:
import QuantLib as ql
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## Market Data and Model Setup

We set up simple constant term structures for risk-free rate and volatility. As inputs we use the parameters from our Black-Scholes example.

In [ ]:
r = 0.0500
sigma = 0.30
S0 = 1.0

### YieldTermStructure, BlackVolTermStructure and BlackScholesProcess

In [ ]:
discountYts  = ql.FlatForward(0, ql.NullCalendar(), r, ql.Actual365Fixed())
discountYtsH = ql.YieldTermStructureHandle(discountYts)
#
volTs  = ql.BlackConstantVol(0, ql.NullCalendar(), sigma, ql.Actual365Fixed())
volTsH = ql.BlackVolTermStructureHandle(volTs)
#
asset = ql.QuoteHandle(ql.SimpleQuote(S0))
process = ql.BlackScholesProcess(asset, discountYtsH, volTsH)

## American Option Pricing

In this section we set up the pricing for American call and put options and compare results to the corresponding European option prices.

In [ ]:
today = ql.Settings.getEvaluationDate(ql.Settings.instance())
print(today)
#
T = 1.4
numberOfDays = int(np.round(T * 365))
print(numberOfDays)
#
expiryDate = today + numberOfDays
print(expiryDate)

European and American options in QuantLib are distinguished by the Exercise object.

In [ ]:
europeanExercise = ql.EuropeanExercise(expiryDate)
americanExercise = ql.AmericanExercise(today, expiryDate)

### Payoffs

In [ ]:
putStrikes  = [ 0.10, 0.30, 0.50, 0.70, 0.90, 1.10, 1.30, 1.50, 1.70, 1.90 ]
callStrikes = [ 0.10, 0.30, 0.50, 0.70, 0.90, 1.10, 1.30, 1.50, 1.70, 1.90 ]
#
putPayoffs  = [ ql.PlainVanillaPayoff(ql.Option.Put,K)  for K in putStrikes  ]
callPayoffs = [ ql.PlainVanillaPayoff(ql.Option.Call,K) for K in callStrikes ]

### Instruments

In [ ]:
europeanPuts  = [ ql.EuropeanOption(p,europeanExercise) for p in putPayoffs  ]
europeanCalls = [ ql.EuropeanOption(p,europeanExercise) for p in callPayoffs ]
#
americanPuts  = [ ql.VanillaOption(p,americanExercise) for p in putPayoffs  ]
americanCalls = [ ql.VanillaOption(p,americanExercise) for p in callPayoffs ]

### CRR Model Pricing

The BinomialCRRVanillaEngine can handle European and American exercise.

In [ ]:
N = 10
crrEngine = ql.BinomialCRRVanillaEngine(process, N)

In [ ]:
for i in europeanPuts + europeanCalls + americanPuts + americanCalls:
    i.setPricingEngine(crrEngine)

We calculate all the prices.

In [ ]:
europeanPutTable = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(putStrikes, europeanPuts):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    europeanPutTable = europeanPutTable.append({ c : v for c,v in zip(europeanPutTable.columns,res) }  , ignore_index=True)
print('European Puts:')
print(europeanPutTable)
#
europeanCallTable = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(callStrikes, europeanCalls):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    europeanCallTable = europeanCallTable.append({ c : v for c,v in zip(europeanCallTable.columns,res) }  , ignore_index=True)
print('European Calls:')
print(europeanCallTable)
#
americanPutTable = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(putStrikes, americanPuts):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    americanPutTable = americanPutTable.append({ c : v for c,v in zip(americanPutTable.columns,res) }  , ignore_index=True)
print('American Puts:')
print(americanPutTable)
#
americanCallTable = pd.DataFrame(columns=('Strike', 'Price', 'Delta', 'Gamma', 'Theta'))
for K, i in zip(callStrikes, americanCalls):
    res = ( K, i.NPV(), i.delta(), i.gamma(), i.theta() )
    americanCallTable = americanCallTable.append({ c : v for c,v in zip(americanCallTable.columns,res) }  , ignore_index=True)
print('American Calls:')
print(americanCallTable)


In [ ]:
def plotResults(resString):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=putStrikes,  y=europeanPutTable[resString],   name='European put',  line=dict(color='royalblue', dash=None) ))
    fig.add_trace(go.Scatter(x=callStrikes, y=europeanCallTable[resString],  name='European Call',  line=dict(color='firebrick', dash=None) ))
    fig.add_trace(go.Scatter(x=putStrikes,  y=americanPutTable[resString],  name='American Put', line=dict(color='royalblue', dash='dash') ))
    fig.add_trace(go.Scatter(x=callStrikes, y=americanCallTable[resString], name='American Call', line=dict(color='firebrick', dash='dash') ))
    fig.update_layout(
        title='Black-Scholes and CRR Model %s, T=%.2f' % (resString,T),
        xaxis_title="Strike K",
        yaxis_title=resString,
        width=600, height=400, autosize=False,
        #margin=dict(l=65, r=50, b=65, t=90),
    )
    fig.show()
    return

plotResults('Price')
plotResults('Delta')
plotResults('Gamma')
plotResults('Theta')

## Continuation and Stopping Region

We set up a function that calculates put option prices for given underlying and time to maturity.

In [ ]:
def putPrice(timeToMaturity, assetPrices, strike=1.0, N=10):
    expiryDate = today + int(np.round(timeToMaturity * 365))
    europeanExercise = ql.EuropeanExercise(expiryDate)
    americanExercise = ql.AmericanExercise(today, expiryDate)
    putPayoff        = ql.PlainVanillaPayoff(ql.Option.Put,strike)
    europeanPut      = ql.VanillaOption(putPayoff,europeanExercise)
    americanPut      = ql.VanillaOption(putPayoff,americanExercise)
    S0 = ql.RelinkableQuoteHandle(ql.SimpleQuote(strike))
    process = ql.BlackScholesProcess(S0, discountYtsH, volTsH)
    engine = ql.BinomialCRRVanillaEngine(process, N)
    europeanPut.setPricingEngine(engine)
    americanPut.setPricingEngine(engine)
    #
    europeanPrices = np.zeros(len(assetPrices))
    americanPrices = np.zeros(len(assetPrices))
    for k, S in enumerate(assetPrices):
        S0.linkTo(ql.SimpleQuote(S))
        europeanPrices[k] = europeanPut.NPV()
        americanPrices[k] = americanPut.NPV()
    intrinsicValue = np.maximum(strike - assetPrices, 0.0)
    return americanPrices, europeanPrices, intrinsicValue

x = np.linspace(0.40, 1.50, 50)
T = np.linspace(0.10, 1.40, 28)

U = np.array([ putPrice(t, x, N=max(2,int(np.round(10*t))))[0] for t in T ])
H = np.array([ putPrice(t, x, N=max(2,int(np.round(10*t))))[2] for t in T ])

t = T[-1] - T # T is time to maturity

In [ ]:
fig = go.Figure(data=[
    go.Surface(x=x, y=t, z=U),
    go.Surface(x=x, y=t, z=H, colorscale='Greys'),
])
fig.update_layout(
    title='CRR American Put Price',
    scene = dict(
        xaxis = dict(
            title = 'x',
        ),
        yaxis = dict(
            title = 't',
        ),
        zaxis = dict(
            title = 'Price',
        ),
    ),
    width=1200, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()